In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [ ]:
import missingno as msno
msno.bar(tot16)
msno.heatmap(tot16)

---

### ignore

In [ ]:
# fixed per year
agricensus = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/agricensus/2020.geojson')

In [ ]:
def loader(a, b, c, d, e):
    istruzione = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/ISTRUZIONE/{a}.geojson')
    occupazione = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/OCCUPAZIONE/{b}.geojson')
    ifc= gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/IFC/{c}.geojson')
    popolazione = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/popolazione/{d}.geojson')
    redditi = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/redditi/{e}.geojson')

    return istruzione, occupazione, ifc, popolazione, redditi


def find_neighbors(gdf, row):
    neighbors = []
    for idx, row2 in gdf.iterrows():
        if (row2['geometry'].touches(row['geometry'])):
            neighbors.append(idx)
    return neighbors


def fill_missing_value(gdf, row, var):
    if pd.isna(row[var]):  # If it is missing...
        neighbors = find_neighbors(gdf, row)
        if neighbors:
            neighbor_val = gdf.loc[neighbors, var]
            # Calculate the average of neighbors (ignoring NaN values)
            if str(gdf[var].dtype) in ['float64']:
                return neighbor_val.mean()
            else:
                m = neighbor_val.mode()
                if m.shape[0] >= 1:
                    return m[0]
    return row[var]

def fix_series_values(row, var):
    if type(row[var]) == pd.core.series.Series:
        return pd.Series(row[var]).iloc[0][0]
    else:
        return row[var]
    
def yearly_data_processing(year, a, b, c, d, e):
    i18, o18, ifc18, p16, r16 = loader(a, b, c, d, e)
    tot16 = i18.merge(o18, how='outer', on='geometry'
                  ).merge(ifc18, how='outer', on='geometry'
                          ).merge(p16, how='outer', on='geometry'
                                  ).merge(r16, how='outer', on='geometry')
    
    tot16.drop(columns=tot16.columns[tot16.isna().all()], inplace=True)
    
    for col in tqdm(tot16.columns, desc='Fixing series values...'):
        tot16.loc[:,col] = tot16[[col]].apply(lambda x: fix_series_values(x, col), axis=1)

    track = 0
    while(tot16.isna().sum().any() & track < 15):
        for col in tqdm(tot16.columns[tot16.isna().any()], desc=f'Fixing missing values time {track}...'):
            tot16.loc[:,col] = tot16[[col, 'geometry']].apply(lambda x: fill_missing_value(tot16, x, col), axis=1)
        track += 1
    
    save_clean_data(tot16, f'per_comune_{year}_provvisorio', '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune')

In [ ]:
for year, lst in param_grid.items():
    yearly_data_processing(year, lst[0], lst[1], lst[2], lst[3], lst[4])

---

### further processing

In [ ]:
years = dict()
for y in range(2016, 2023):
    years[y] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/per_comune_{y}_provvisorio.geojson')

years[20232024] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/per_comune_20232024_provvisorio.geojson')

In [ ]:
columns_list = list(years[2016].columns)
for y, df in years.items():
    for c in df.columns:
        if c not in columns_list:
            print(f'Column {c} appears in year {y} and not in 2016')

In [ ]:
for gdf in years.values():
    for col in ['IFC (decile)','AMB_01', 'AMB_02','AMB_03', 'ECO_01', 'ECO_02','SOC_01','SOC_02','SOC_03','SOC_04','TER_01','TER_02','TER_03']:
        for j in gdf[col].index:
            c = gdf.loc[j, col]
            if isinstance(c, str):
                gdf.loc[j, col] = float(c.replace('.', '').replace(',', '.'))
        gdf[col] = gdf[col].astype('float64')

In [ ]:
def find_neighbors(gdf, row):
    neighbors = []
    for idx, row2 in gdf.iterrows():
        if (row2['geometry'].touches(row['geometry'])):
            neighbors.append(idx)
    return neighbors


def fill_missing_value(gdf, row, var):
    if pd.isna(row[var]):  # If it is missing...
        neighbors = find_neighbors(gdf, row)
        if neighbors:
            neighbor_val = gdf.loc[neighbors, var]
            # Calculate the average of neighbors (ignoring NaN values)
            if str(gdf[var].dtype) in ['float64']:
                return neighbor_val.mean()
            else:
                m = neighbor_val.mode()
                if m.shape[0] >= 1:
                    return m[0]
    return row[var]

In [ ]:
for gdf in years.values():
    gdf.loc[:,'Fascia demografica'] = gdf[['Fascia demografica', 'geometry']].apply(lambda x: fill_missing_value(gdf, x, 'Fascia demografica'), axis=1)

In [ ]:
columns_list = list(years[2016].columns)
for y, df in years.items():
    for c in df.columns:
        if c not in columns_list:
            print(f'Column {c} appears in year {y} and not in 2016')

In [ ]:
for gdf in years.values():
    if '65_studente/ssa' in gdf.columns:
        gdf.drop(columns='65_studente/ssa', inplace=True)

In [ ]:
for y, gdf in years.items():
    gdf.to_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/per_comune_{y}_provvisorio.geojson', index=False, driver='GeoJSON')

In [ ]:
from shapely.geometry import Point